In [1]:
! pip install semantic-kernel -U 
! pip install qdrant_client -U 
! pip install pyautogen -U
! pip install youtube_transcript_api -U

In [2]:
import uuid,json
from youtube_transcript_api import YouTubeTranscriptApi

In [3]:
import autogen
import semantic_kernel as sk
from dotenv import load_dotenv
from autogen.cache import Cache
import semantic_kernel.connectors.ai.open_ai as skaoai
from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory

In [4]:
def saveblog(blog):
    file = './blogs/blog-'+ str(uuid.uuid4()) +'.md'
    with open(file, 'w') as f:
        f.write(blog)
    return file

In [5]:
def downloadandsaveyoutubecaption(videourl):
    video_id = videourl.split('=')[1]
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
    save_path='./data/captions/captions-'+str(uuid.uuid4())+'.txt'

    with open(save_path, 'w') as f:
        for entry in transcript:
            f.write(f"{entry['text']}\n")

    return save_path 

In [6]:
async def savedtovectordb(captionfile):
    kernel = sk.Kernel()
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_service(skaoai.AzureChatCompletion("GPT4Model",endpoint,api_key=api_key,api_version = "2023-12-01-preview"))
    kernel.add_service(skaoai.AzureTextEmbedding("EmbeddingModel", endpoint,api_key=api_key,api_version = "2023-12-01-preview"))
    base_plugin = "./plugins"
    files_plugin = kernel.import_plugin_from_prompt_directory(base_plugin , "FilePlugin")
    file = open(captionfile, "r") 
    content = file.read()
    captionsFunc = files_plugin["Captions"]
    result = await kernel.invoke(captionsFunc, sk.KernelArguments(input=content)) 
    # print(result.result) 
    kblist = [] 
    # print(result.result.replace("\n", "x"))
    json_result = json.loads(str(result).replace("\n", "").replace("System:",""))
    for item in json_result:
        print(item)
        kblist.append(item)
    base_vectordb = 'kb-vectordb'
    qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
    await qdrant_store.create_collection(base_vectordb)
    memory = SemanticTextMemory(storage=qdrant_store, embeddings_generator=skaoai.AzureTextEmbedding("EmbeddingModel", endpoint,api_key=api_key,api_version = "2023-12-01-preview"))

    # kernel.register_memory_store(memory_store=qdrant_store)
    for item in kblist:
        content = item["kb"] + ' - ' + item["content"]
        id =str(uuid.uuid4())
        await memory.save_information(base_vectordb, id=id, text=content)
    file.close()
    return "saved"

In [7]:
async def translateblog(blogfile):
    content = open(blogfile, "r").read()
    kernel = sk.Kernel()
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_service(skaoai.AzureChatCompletion("GPT4Model",endpoint,api_key=api_key,api_version = "2023-12-01-preview"))
    base_plugin = "./plugins/"
    file_plugin = kernel.import_plugin_from_prompt_directory(base_plugin , "FilePlugin")
    translateFunc = file_plugin["Translate"]
    result = await kernel.invoke(translateFunc, sk.KernelArguments(input=content)) 
    save_path='./blogs/zh-blogs-'+str(uuid.uuid4())+'.md'
    with open(save_path, 'w') as f:
        f.write(str(result))
    
    return "done"

In [8]:
async def writeblog(content):
    kernel = sk.Kernel()
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_service(skaoai.AzureChatCompletion("GPT4Model",endpoint,api_key=api_key,api_version = "2023-12-01-preview"))
    base_plugin = "./plugins/"
    write_plugin = kernel.import_plugin_from_prompt_directory(base_plugin , "WritePlugin")
    blogFunc = write_plugin["Blogs"]
    result = await kernel.invoke(blogFunc, sk.KernelArguments(input=content)) 
    return str(result)

In [9]:
# @user_proxy.register_for_execution()
async def ask(question):
    kernel = sk.Kernel()
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_service(skaoai.AzureChatCompletion("GPT4Model",endpoint,api_key=api_key,api_version = "2023-12-01-preview"))


    kernel.add_service(skaoai.AzureTextEmbedding("EmbeddingModel", endpoint,api_key=api_key,api_version = "2023-12-01-preview"))
#     kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion("GPT4Model",endpoint,api_key=api_key,api_version = "2023-12-01-preview"))
#     kernel.add_text_embedding_generation_service(
#             "embeddings_services", skaoai.AzureTextEmbedding("EmbeddingModel", endpoint,api_key=api_key,api_version = "2023-12-01-preview")
#     )
    base_plugin = "./plugins"
    answer_plugin = kernel.import_plugin_from_prompt_directory(base_plugin , "AnswerPlugin")
    qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
    memory = SemanticTextMemory(storage=qdrant_store, embeddings_generator=skaoai.AzureTextEmbedding("EmbeddingModel", endpoint,api_key=api_key,api_version = "2023-12-01-preview"))

    # kernel.register_memory_store(memory_store=qdrant_store)

    ask = question
    base_vectordb = "kb-vectordb"


    memories = await memory.search(
        base_vectordb, ask, limit=1, min_relevance_score=0.8
    )

    result = ''
    for item in memories:
        print(f"Top Result: {item.text} with score {item.relevance}")
        result = item.text

    answerFunc = answer_plugin["Summary"]

    summary_result = await kernel.invoke(answerFunc, sk.KernelArguments(input=result)) 

    return str(summary_result)

In [10]:
config_list = autogen.config_list_from_json(
    "AOAI_CONFIG_LIST",
    filter_dict={
        "model": ["GPT4Model","GPT3Model"],
    },
)

In [11]:

llm_config={
    "config_list": config_list,
    "functions": [
        {
            "name": "downloadandsaveyoutubecaption",
            "description": "Download youtube caption from human input, save to file and return file path",
            "parameters": {
                "type": "object",
                "properties": {
                    "videourl": {
                        "type": "string",
                        "description": "Youtube video url from human input",
                    }
                },
                "required": ["videourl"],
            },
        },
        {
            "name": "savedtovectordb",
            "description": "get saved caption file and save it knowledge to vectordb",
            "parameters": {
                "type": "object",
                "properties": {
                    "captionfile": {
                        "type": "string",
                        "description": "return saved caption file path from download youtbe caption function",
                    }
                },
                "required": ["captionfile"],
            },
        },
        {
            "name": "ask",
            "description": "ask question about Machine Learning,  get basic knowledge",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "About Machine Learning",
                    }
                },
                "required": ["question"],
            },
        },
        {
            "name": "writeblog",
            "description": "write blogs in markdown format",
            "parameters": {
                "type": "object",
                "properties": {
                    "content": {
                        "type": "string",
                        "description": "basic content",
                    }
                },
                "required": ["content"],
            },
        },
        {
            "name": "saveblog",
            "description": "save blogs",
            "parameters": {
                "type": "object",
                "properties": {
                    "blog": {
                        "type": "string",
                        "description": "basic content",
                    }
                },
                "required": ["blog"],
            },
        },
        {
            "name": "translateblog",
            "description": "translate blog to Chinese",
            "parameters": {
                "type": "object",
                "properties": {
                    "blogfile": {
                        "type": "string",
                        "description": "blog file path",
                    }
                },
                "required": ["blogfile"],
            },
        }
    ],
}

In [12]:
collect_kb_assistant = autogen.AssistantAgent(
    name="collectkbassistant",
    llm_config=llm_config,
    system_message="You are a knowledge collector assistant , download and save youtube caption from youtube url, save it as knowledge to vectordb from return saved caption file path ",
)

blog_editor_assistant = autogen.AssistantAgent(
    name="blogeditorassistant",
    llm_config=llm_config,
    system_message="You are a blog editor assistant,  write an introduction based on answer from questions . After preparing these basic materials, write a blog and save this blog to local file",
)

translate_assistant = autogen.AssistantAgent(
    name="transalteassistant",
    llm_config=llm_config,
    system_message="You are a translate assistant , translate  content to Chinese from blog file path",
)

In [13]:

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=30,
    code_execution_config=False
)


user_proxy.register_function(
    function_map={
        "downloadandsaveyoutubecaption": downloadandsaveyoutubecaption,
        "savedtovectordb": savedtovectordb,
        "ask": ask,
        "writeblog": writeblog,
        "saveblog": saveblog,
        "translateblog": translateblog
    }
)

In [14]:
# from autogen import GroupChatManager

In [15]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, collect_kb_assistant, blog_editor_assistant,translate_assistant], messages=[],max_round=30)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config={'config_list': config_list})

In [16]:
await user_proxy.a_initiate_chat(
    manager,
    message="""
            Use this link https://www.youtube.com/watch?v=1qs6QKk0DVc as knowledge with collect knowledge assistant. Find the answers to the 3 questions below to write blog  and save and save this blog to local file with blog editor assistant. And translate this blog to Chinese with translate assistant.

            1. What is GitHub Copilot ?
            2. How to Install GitHub Copilot ?
            3. Limitations of GitHub Copilot

           Let's go
"""
    )

Admin (to chat_manager):


            Use this link https://www.youtube.com/watch?v=1qs6QKk0DVc as knowledge with collect knowledge assistant. Find the answers to the 3 questions below to write blog  and save and save this blog to local file with blog editor assistant. And translate this blog to Chinese with translate assistant.

            1. What is GitHub Copilot ?
            2. How to Install GitHub Copilot ?
            3. Limitations of GitHub Copilot

           Let's go


--------------------------------------------------------------------------------
collectkbassistant (to chat_manager):

***** Suggested function Call: downloadandsaveyoutubecaption *****
Arguments: 
{
"videourl": "https://www.youtube.com/watch?v=1qs6QKk0DVc"
}
******************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION downloadandsaveyoutubecaption...
Admin (to chat_manager):

***** Response

ChatResult(chat_id=None, chat_history=[{'content': "\n            Use this link https://www.youtube.com/watch?v=1qs6QKk0DVc as knowledge with collect knowledge assistant. Find the answers to the 3 questions below to write blog  and save and save this blog to local file with blog editor assistant. And translate this blog to Chinese with translate assistant.\n\n            1. What is GitHub Copilot ?\n            2. How to Install GitHub Copilot ?\n            3. Limitations of GitHub Copilot\n\n           Let's go\n", 'role': 'assistant'}, {'content': '', 'function_call': {'arguments': '{\n"videourl": "https://www.youtube.com/watch?v=1qs6QKk0DVc"\n}', 'name': 'downloadandsaveyoutubecaption'}, 'name': 'collectkbassistant', 'role': 'assistant'}, {'content': './data/captions/captions-87dd1902-d487-4ba7-a07f-538c9ec44485.txt', 'name': 'downloadandsaveyoutubecaption', 'role': 'function'}, {'content': '', 'function_call': {'arguments': '{\n"captionfile": "./data/captions/captions-87dd1902-d48